In [1]:
import pandas as pd
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import datetime
import time

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [3]:
#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc.h5'
# oading up our datasets
train_data_dir = 'data3/train' 
validation_data_dir = 'data3/val' 
test_data_dir = 'data3/test'
 
# number of epochs to train top model 
epochs = 3 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 16 

In [4]:
#Loading vgc16 model
vgg16 = applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255) 
#needed to create the bottleneck .npy files

In [5]:
'''
#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) 
 
np.save('bottleneck_features_train.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)
'''

"\n#__this can take an hour and half to run so only run it once. \n#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__\nstart = datetime.datetime.now()\n \ngenerator = datagen.flow_from_directory( \n    train_data_dir, \n    target_size=(img_width, img_height), \n    batch_size=batch_size, \n    class_mode=None, \n    shuffle=False) \n \nnb_train_samples = len(generator.filenames) \nnum_classes = len(generator.class_indices) \n \npredict_size_train = int(math.ceil(nb_train_samples / batch_size)) \n \nbottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) \n \nnp.save('bottleneck_features_train.npy', bottleneck_features_train)\nend= datetime.datetime.now()\nelapsed= end-start\nprint ('Time: ', elapsed)\n"

In [6]:
#training data
generator_top = datagen.flow_from_directory( 
   train_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 58394 images belonging to 77 classes.


In [7]:
#validation data
generator_val = datagen.flow_from_directory( 
   validation_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
validation_data = np.load('bottleneck_features_train.npy') 
 
# get the class labels for the training data, in the original order 
validation_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

Found 19465 images belonging to 76 classes.


In [8]:
#test data
generator_test = datagen.flow_from_directory( 
   test_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
test_data = np.load('bottleneck_features_train.npy') 
 
# get the class labels for the training data, in the original order 
test_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
test_labels = to_categorical(test_labels, num_classes=num_classes)

Found 19465 images belonging to 76 classes.


In [ ]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
#start = datetime.datetime.now()
model = Sequential() 
model.add(Flatten(input_shape=train_data.shape[1:])) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.5)) 
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.RMSprop(lr=1e-4),
   metrics=['acc'])
history = model.fit(train_data, train_labels, 
   epochs=3,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("[INFO] Loss: {}".format(eval_loss)) 
#end= datetime.datetime.now()
#elapsed= end-start
#print ('Time: ', elapsed)

Epoch 1/3
3650/3650 [==============================] - 39s 11ms/step - loss: 3.6980 - acc: 0.1734 - val_loss: 3.1852 - val_acc: 0.2112
Epoch 2/3
3650/3650 [==============================] - 33s 9ms/step - loss: 3.3096 - acc: 0.2125 - val_loss: 3.0772 - val_acc: 0.2518
Epoch 3/3
3650/3650 [==============================] - 32s 9ms/step - loss: 3.2220 - acc: 0.2241 - val_loss: 3.0233 - val_acc: 0.2555
